# 🧪 Lab Manual: Module 7 – Web Search
**Course:** Information Retrieval and Web Search  
**Topics Covered:**
- Detecting duplicates and near duplicates using shingling
- Detecting web spam in IR and web search

---

## <font color='red'>Submission: Submit both .ipynb file and .ipynb converted to PDF</font>
## <font color='blue'>Submissions with following cases will get a zero</font>
* ## <font color='red'>Code or commented text truncated from the pdf version of the notebook</font>
* ### <font color='blue'>Any compilation error in the notebook</font>
* ### <font color='blue'>Missing output for any of the programming cells. There should be an output for every code cell</font>
---

## Part 1: Detecting Duplicates and Near Duplicates Using Shingling

---

### Q1. Document Preparation
Load a set of 4–6 short sample documents/web pages (e.g., product descriptions, blog excerpts).  
- Tokenize and normalize the text.  
- Preprocess with necessary steps before shingling?  
- Use `nltk.word_tokenize()` and lowercase normalization.

---

In [1]:
# Using wikipedia articles for product descriptions
import wikipedia

titles = ["iPhone 7","Tesla Cybertruck","Tesla Model s",
          "PlayStation 5","iPhone 6", "Playstation 4"]
documents = {}
for title in titles:
    page = wikipedia.page(title)
    documents[title.lower()] = page.summary
documents

{'iphone 7': 'The iPhone 4 is a smartphone that was developed and marketed by Apple. It is the fourth generation of the iPhone lineup, succeeding the iPhone 3GS and preceding the iPhone 4s. Following a number of notable leaks, the iPhone 4 was first unveiled on June 7, 2010, at Apple\'s Worldwide Developers Conference in San Francisco, and was released on June 24, 2010, in the United States, United Kingdom, France, Germany, and Japan.\nThe iPhone 4 introduced a new hardware design to the iPhone family, which Apple\'s CEO Steve Jobs touted as the thinnest smartphone in the world at the time; it consisted of a stainless steel frame that doubled as an antenna, with internal components situated between two panels of aluminosilicate glass. The iPhone 4 introduced Apple\'s new high-resolution "Retina display" (with a pixel density of 326 pixels per inch) while maintaining the same physical size and aspect ratio as its predecessors, Apple\'s A4 system-on-chip, and with iOS 4—which notably int

In [2]:
# TOKENIZING AND NORMALIZING DOCUMENTS
import nltk
tokenized_normalized = {}
for title, summary in documents.items():
    tokenized_doc = nltk.word_tokenize(summary)
    lowercased_doc = [token.lower() for token in tokenized_doc]
    normalized_doc = [
        token for token in lowercased_doc if token.isalpha() and token.isascii()
    ]
    tokenized_normalized[title] = normalized_doc
print(tokenized_normalized)

{'iphone 7': ['the', 'iphone', 'is', 'a', 'smartphone', 'that', 'was', 'developed', 'and', 'marketed', 'by', 'apple', 'it', 'is', 'the', 'fourth', 'generation', 'of', 'the', 'iphone', 'lineup', 'succeeding', 'the', 'iphone', 'and', 'preceding', 'the', 'iphone', 'following', 'a', 'number', 'of', 'notable', 'leaks', 'the', 'iphone', 'was', 'first', 'unveiled', 'on', 'june', 'at', 'apple', 'worldwide', 'developers', 'conference', 'in', 'san', 'francisco', 'and', 'was', 'released', 'on', 'june', 'in', 'the', 'united', 'states', 'united', 'kingdom', 'france', 'germany', 'and', 'japan', 'the', 'iphone', 'introduced', 'a', 'new', 'hardware', 'design', 'to', 'the', 'iphone', 'family', 'which', 'apple', 'ceo', 'steve', 'jobs', 'touted', 'as', 'the', 'thinnest', 'smartphone', 'in', 'the', 'world', 'at', 'the', 'time', 'it', 'consisted', 'of', 'a', 'stainless', 'steel', 'frame', 'that', 'doubled', 'as', 'an', 'antenna', 'with', 'internal', 'components', 'situated', 'between', 'two', 'panels', 'of

### Q2. Shingle Generation
Generate k-shingle sets.  
- Choose a value for k (e.g., 3 or 4).  
- Generate shingles for each document using consecutive sequences of k terms.  
- Explain how the choice of k affects sensitivity to small changes.

---

In [3]:
# Shingle fuction, default as k=3
def shingle(document_tokens, k=3):
    shingles = []
    for idx in range(len(document_tokens) - k +1):
        shingle = tuple(document_tokens[idx:idx+k])
        shingles.append(shingle)
    return shingles

In [4]:
# shingle every document and save in dict
shingled_docs = {}
for title, doc in tokenized_normalized.items():
    shingled_doc = shingle(doc)
    shingled_docs[title] = shingled_doc
print(shingled_docs)

{'iphone 7': [('the', 'iphone', 'is'), ('iphone', 'is', 'a'), ('is', 'a', 'smartphone'), ('a', 'smartphone', 'that'), ('smartphone', 'that', 'was'), ('that', 'was', 'developed'), ('was', 'developed', 'and'), ('developed', 'and', 'marketed'), ('and', 'marketed', 'by'), ('marketed', 'by', 'apple'), ('by', 'apple', 'it'), ('apple', 'it', 'is'), ('it', 'is', 'the'), ('is', 'the', 'fourth'), ('the', 'fourth', 'generation'), ('fourth', 'generation', 'of'), ('generation', 'of', 'the'), ('of', 'the', 'iphone'), ('the', 'iphone', 'lineup'), ('iphone', 'lineup', 'succeeding'), ('lineup', 'succeeding', 'the'), ('succeeding', 'the', 'iphone'), ('the', 'iphone', 'and'), ('iphone', 'and', 'preceding'), ('and', 'preceding', 'the'), ('preceding', 'the', 'iphone'), ('the', 'iphone', 'following'), ('iphone', 'following', 'a'), ('following', 'a', 'number'), ('a', 'number', 'of'), ('number', 'of', 'notable'), ('of', 'notable', 'leaks'), ('notable', 'leaks', 'the'), ('leaks', 'the', 'iphone'), ('the', 'iph

#### ANSWER

**Smaller k values are more sensitive in shingling because it requires less consecutive words to match. Larger k values make it less sensitive because more back to back words have to match.**

### Q3. Jaccard Similarity Computation
Compute the Jaccard similarity between all pairs of documents.  
- Formula:  

$$
J(A, B) = \frac{|A \cap B|}{|A \cup B|}
$$

### Interpretation
Measures overlap between shingle sets
* **$J(A, B)$** ranges from **0 to 1**.
* A value of **1** means the sets $A$ and $B$ are identical.
* A value of **0** means the sets $A$ and $B$ have no elements in common.
---

In [5]:
# Jaccard Similarity Computation function
def jaccard(doc1_shingles, doc2_shingles):
    # Jaccard uses unqiue sets
    doc1_shingles = set(doc1_shingles)
    doc2_shingles = set(doc2_shingles)

    union = len(doc1_shingles | doc2_shingles)
    intersection = len(doc1_shingles & doc2_shingles)

    if intersection/union > 0:
        return intersection/union
    else:
        return 0

In [6]:
# calculate jaccard between each pair of docs
for title1, doc1 in shingled_docs.items():
    for title2, doc2 in shingled_docs.items():
        if title1 <= title2:
            print(f"*{title1}* and *{title2}*: {round(jaccard(doc1,
                                                        doc2),4)}")

*iphone 7* and *iphone 7*: 1.0
*iphone 7* and *tesla cybertruck*: 0.0061
*iphone 7* and *tesla model s*: 0.0015
*iphone 7* and *playstation 5*: 0.0112
*iphone 7* and *playstation 4*: 0.0046
*tesla cybertruck* and *tesla cybertruck*: 1.0
*tesla cybertruck* and *tesla model s*: 0
*tesla model s* and *tesla model s*: 1.0
*playstation 5* and *tesla cybertruck*: 0.0025
*playstation 5* and *tesla model s*: 0.0017
*playstation 5* and *playstation 5*: 1.0
*iphone 6* and *iphone 7*: 0.0422
*iphone 6* and *tesla cybertruck*: 0
*iphone 6* and *tesla model s*: 0
*iphone 6* and *playstation 5*: 0.01
*iphone 6* and *iphone 6*: 1.0
*iphone 6* and *playstation 4*: 0.0041
*playstation 4* and *tesla cybertruck*: 0.0023
*playstation 4* and *tesla model s*: 0
*playstation 4* and *playstation 5*: 0.0399
*playstation 4* and *playstation 4*: 1.0



### Q4. Thresholding for Near Duplicates
Choose a similarity threshold (e.g., 0.8 or 0.9).  
- Identify which document pairs exceed this threshold.  
- Justify your threshold choice.  
- Flag near duplicates accordingly.

---

#### ANSWER

**Our threshold is 0.5, because anything above this threshold means that the documents are more than 50% similar.**

**As we can see in our documents above, no document pairs exceed this threshold. This is probably because these wikipedia product descriptions have a lot of text, thus creating a lot of shingles. There are probably not many shingle matches in each document pairing due to that factor.**

**Another thing to consider is the word sensitivities, for example a shingle can be the same structurally, but the difference in "iphone 6" and "iphone 7" won't allow the intersection since the numbers are different.**

**The closest documents are "iphone 6" and "iphone 7" which have a score of 0.0422 (or 4.22%). This makes sense because these two iphone models are very similar.**

### Q5. Reflection
- What are the strengths and limitations of shingling for duplicate detection?  
- How would you handle documents with minor formatting differences?  
- Could shingling be extended to multimedia content?

**Instructions:**

- Use headings or bold labels
- Use markdown formatting to keep your notebook organized and readable.

---

#### ANSWER

**Shingling is very good with being sensitive to major changes (depending on k), and it can be scaled to large documents since we are comparing k-sized lists in each document to retrieve a score.**

**It has a weakness of not being able to recognize sometimes semantically correct or differently phrased pairings. As mentioned above, the slight difference between "iphone6" and "iphone7" can lower the score significantly, even though they are very similar contextually.**

**Techniques such as synonym expansion may help with minor formatting differences, as we can expand the vocabulary but keep the context of the sentences the same. This approach would create a more accurate and advanced score compared to what we have now.**

**Shingling could most likely be used in multimedia content since the logic does not change of the algorithm. In images, we can break down pixels by k sized sets and find a similarity score of 2 pictures, same with audio trimmings.**

## Part 2: Detecting Web Spam in IR and Web Search

---

### Q6. Keyword Stuffing Detection
Write a Python function that detects keyword stuffing in a document (web page).  
- Accept a string of text and a target keyword.  
- Count the number of times the keyword appears.  
- Flag the document if the keyword frequency exceeds a threshold (e.g., 5).  
- Normalize the text before counting.

  
## Example Input:

`
text = "Buy Maui golf real estate, Maui golf homes, Maui golf villas..."
`
`
keyword = "Maui"
`

## Expected Output:

`
True  # Keyword stuffing detected
`

---

In [7]:
def keyword_stuff(text, keyword):
    keyword = keyword.lower()
    # Normalize and tokenize
    tokenized_text = nltk.word_tokenize(text)
    lowercased_text = [token.lower() for token in tokenized_text]
    normalized_text = [
        token for token in lowercased_text if token.isalpha() and token.isascii()
    ]
    occurence = 0
    for word in normalized_text:
        if word == keyword:
            occurence += 1
    if occurence >= 5:
        print("Keyword stuffing detected")
        return True
    else:
        return False

In [8]:
keyword_stuff("""Buy Maui golf real estate, Maui golf homes, Maui golf villas,
                Maui resorts,Maui soccer, Maui""","Maui")

Keyword stuffing detected


True

In [9]:
keyword_stuff("Buy Maui golf real estate, Maui golf homes, Maui golf villas, Maui resorts","Maui")

False

### Q7. Hidden Text Detection
Simulate and detect hidden text in HTML using CSS-based concealment.  
- Accept an HTML string.  
- Search for patterns like `style="color:white"` or `display:none`.  
- Return True if hidden text is detected.

## Example Input:

`
html = '<div style="color:white">Hidden keyword</div>'
`


## Expected Output:

`
True 
`

---

In [10]:
def hidden_text(html):
    html = html.lower()

    hidden_text = [
        'display: none',
        'display:none',
        'color:white',
        'color: white',
        'color:#ffffff',
        'color: #ffffff',
        'font-size:0',
        'font-size: 0',
        'opacity:0',
        'opacity: 0'
    ]

    for text in hidden_text:
        if text in html:
            return True
    return False

In [11]:
hidden_text('<div style="color:white">Hidden keyword</div>')

True

In [12]:
hidden_text('<div style="color:black">Hidden keyword</div>')

False

### Q8. Cloaking Detection
Compare crawler and user views of a webpage to detect cloaking.  
- Accept two strings: one for crawler view, one for user view.  
- Normalize both views.  
- Return True if the views differ significantly.

## Example Input:

`
crawler_view = "Buy cheap flights to Rome"
user_view = "Welcome to our travel blog!"
`


## Expected Output:

`
True  # Cloaking detected
`

---

In [13]:
def cloaking(crawler, user):
    crawler = crawler.lower()
    # Normalize and tokenize
    tokenized_crawler = nltk.word_tokenize(crawler)
    lowercased_crawler = [token.lower() for token in tokenized_crawler]
    normalized_crawler = [
        token for token in lowercased_crawler if token.isalpha() and token.isascii()
    ]

    user = user.lower()
    # Normalize and tokenize
    tokenized_user = nltk.word_tokenize(user)
    lowercased_user = [token.lower() for token in tokenized_user]
    normalized_user = [
        token for token in lowercased_user if token.isalpha() and token.isascii()
    ]

    # Shingling both user and crawler
    crawler_shingle = shingle(normalized_crawler)
    user_shingle = shingle(normalized_user)

    # Jaccard similarity
    similarity = jaccard(crawler_shingle, user_shingle)
    # Let's use threshold of 0.5 since 50% similarity is good 
    if similarity < 0.5:
        print("Cloaking Detected")
        return True
    return False

In [14]:
cloaking("Buy cheap flights to Rome", "Welcome to our travel blog!")

Cloaking Detected


True

In [15]:
cloaking("Buy cheap flights to Rome", "Buy cheap flights to Italy")

False


### Q9. Doorway Page Detection
Detect doorway pages that redirect users immediately.  
- Accept an HTML string.  
- Search for redirect patterns like `<meta http-equiv='refresh'>` or `window.location.href`.  
- Return True if a redirect is found.

## Example Input:

`
html = "<meta http-equiv='refresh' content='0;url=https://ads.com'>"
`


## Expected Output:

`
True 
`

---

In [16]:
def doorway_detect(html):
    html = html.lower()

    doorway_tags = [
        "http-equiv='refresh'",
        'http-equiv="refresh"',
        'http-equiv=refresh',
        "window.location.href"
    ]

    for text in doorway_tags:
        if text in html:
            return True
    return False

In [17]:
doorway_detect("<meta http-equiv='refresh' content='0;url=https://ads.com'>")

True

In [18]:
doorway_detect('<div style="color:white">Hidden keyword</div>')

False

### Q10. Link Spam Detection
Detect link spam based on repeated or artificial inbound links.  
- Accept a list of inbound URLs.  
- Count duplicates and diversity.  
- Return True if the number of unique links is low compared to total links.

## Example Input:

`
links = ["siteA.com", "siteA.com", "siteA.com", "siteB.com"]
`


## Expected Output:

`
True # Link spam detected
`

---

In [19]:
def link_spam(url_list):
    if len(url_list) == 0:
        return False

    # Lowercase first
    links = [url.lower() for url in url_list]
    
    total_urls = len(links)
    unique_urls = len(set(links))

    ratio = unique_urls / total_urls
    # using threshold of 0.5
    if ratio <= 0.5:
        print("Link spam detected")
        return True
    return False

In [20]:
link_spam(["siteA.com", "siteA.com", "siteA.com", "siteB.com"])

Link spam detected


True

In [21]:
link_spam(["siteA.com", "siteB.com", "siteC.com", "siteD.com"])

False

### Q11: Estimate Content Quality of a Web Page

**Objective:**  
Write a Python function that estimates the content quality of a web page based on its visible text. The function should return a score between 0 and 1, where higher values indicate better-written, more informative content.

**Background:**  
Content quality is a key signal in search engine ranking. Pages with rich vocabulary, well-structured sentences, and minimal filler (e.g., stopwords) tend to be more useful to users and rank higher. This task simulates how search engines might heuristically assess page quality.

**Instructions:**

1. **Input:**  
   - Accept a string containing raw HTML content of a web page.

2. **Text Extraction:**  
   - Use `BeautifulSoup` to remove `<script>` and `<style>` tags.
   - Extract visible text and normalize whitespace.

3. **Tokenization and Preprocessing:**  
   - Use `nltk.word_tokenize()` to split text into words.
   - Use `nltk.sent_tokenize()` to split text into sentences.
   - Filter out non-alphabetic tokens and lowercase all words.

4. **Heuristic Scoring:**  
   Compute the following metrics:
   - **Average sentence length**: total words / total sentences  
     (ideal range: 15–25 words per sentence)
   - **Vocabulary richness**: unique words / total words  
     (higher is better)
   - **Stopword ratio**: proportion of stopwords in the text  
     (lower is better)

5. **Final Score:**  
   Combine the metrics using weighted scoring:
   - 40% weight for sentence length (normalized to 0–1)
   - 40% weight for vocabulary richness
   - 20% weight for inverse stopword ratio

6. **Output:**  
   - Return a float score between 0 and 1, rounded to three decimal places.

**Example Usage:**

```python
html_content = "<html><body><h1>Welcome</h1><p>Python is a versatile language used in many domains.</p></body></html>"
score = estimate_webpage_content_quality(html_content)
print("Content Quality Score:", score)
```
---

In [22]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

def estimate_webpage_content_quality(html):
    # Text extraction
    bs = BeautifulSoup(html, 'html.parser')

    for tag in bs(['script', 'style']):
        tag.decompose()

    text = bs.get_text(separator=' ', strip=True)
    text = ' '.join(text.split())

    if not text:
        return 0.0
        
    # Tokenization and preprocessingg
    # lowercasing
    sentences = nltk.sent_tokenize(text)
    text = text.lower()
    words = nltk.word_tokenize(text)

    # filter out non alphabetic tokens
    words = [
        word for word in words if word.isalpha()]

    if len(sentences)==0 or len(words)==0:
        return 0.0

    # scores
    avg_sen_len = len(words)/len(sentences)
    vocab_richness = len(set(words))/len(words)
    stop_words = set(stopwords.words('english'))

    stopword_ratio = sum(1 for word in words if word in stop_words)/len(words)
    
    # Normalize average sentence length
    if avg_sen_len < 15:
        sen_score = avg_sen_len/15
    # Ideal ratio is from 15-25
    elif avg_sen_len <= 25:
        sen_score = 1
    else:
        sen_score = max(0, 1-(avg_sen_len-25)/25)

    # final score (Use 1-stopword_ratio) for the inverse
    score = (0.4*sen_score)+(0.4*vocab_richness)+(0.2*(1-stopword_ratio))
    return round(score,3)

In [23]:
html="<html><body><h1>Welcome</h1><p>Python is a versatile language used in many domains.</p></body></html>"
score = estimate_webpage_content_quality(html)
print("Content Quality Score:", score)

Content Quality Score: 0.807


### Q12: Calculate Keyword Density of a Web Page

**Objective:**  
Write a Python function that calculates the keyword density of a web page. Keyword density is the ratio of the number of times a specific keyword appears to the total number of words in the page’s visible text.

**Background:**  
Keyword stuffing is a common spam tactic where keywords are repeated excessively to manipulate search rankings. Measuring keyword density helps identify whether a page uses keywords naturally or abusively.

**Instructions:**

1. **Input:**  
   - Accept two inputs:
     - `html` (str): Raw HTML content of the web page.
     - `keyword` (str): The keyword to measure.

2. **Text Extraction:**  
   - Use `BeautifulSoup` to remove `<script>` and `<style>` tags.
   - Extract visible text and normalize whitespace.

3. **Tokenization and Preprocessing:**  
   - Use `nltk.word_tokenize()` to split text into words.
   - Filter out non-alphabetic tokens and lowercase all words.

4. **Keyword Density Calculation:**  
   - Count the number of times the keyword appears.
   - Divide by the total number of words.
   - Return the result as a float rounded to three decimal places.

**Example Usage:**

```python
html_content = "<html><body><p>Buy Maui golf real estate, Maui golf homes, Maui golf villas...</p></body></html>"
density = calculate_keyword_density(html_content, keyword="Maui")
print("Keyword Density:", density)
```
---

In [24]:
def calculate_keyword_density(html, keyword):
    # Text extraction
    bs = BeautifulSoup(html, 'html.parser')

    for tag in bs(['script', 'style']):
        tag.decompose()

    text = bs.get_text(separator=' ', strip=True)
    text = ' '.join(text.split())

    if not text:
        return 0.0

    # Tokenization and preprocessingg
    # lowercasing
    text = text.lower()
    words = nltk.word_tokenize(text)

    # filter out non alphabetic tokens
    words = [
        word for word in words if word.isalpha()]

    if len(words)==0:
        return 0.0

    occurrence = 0.0
    for word in words:
        if word == keyword.lower():
            occurrence += 1

    ratio = occurrence / len(words)
    return round(ratio,3)

In [25]:
html="<html><body><p>Buy Maui golf real estate, Maui golf homes, Maui golf villas...</p></body></html>"
density = calculate_keyword_density(html, keyword="Maui")
print("Keyword Density:", density)

Keyword Density: 0.273


### Q13: Calculate Link Diversity of a Web Page

**Objective:**  
Write a Python function that calculates the link diversity of a web page. Link diversity is the ratio of unique outbound links to total outbound links in the page’s HTML.

**Background:**  
Link spam involves creating many artificial or repeated links to falsely boost a page’s authority. Measuring link diversity helps detect unnatural linking behavior.

**Instructions:**

1. **Input:**  
   - Accept one input:
     - `html` (str): Raw HTML content of the web page.

2. **Link Extraction:**  
   - Use `BeautifulSoup` to extract all `<a>` tags with `href` attributes.
   - Filter out internal links (e.g., those starting with `#`, `/`, or the same domain).
   - Normalize and collect all outbound links.

3. **Link Diversity Calculation:**  
   - Count total outbound links.
   - Count unique outbound links.
   - Compute diversity as:  
    $$
    \text{Link Diversity} = \frac{\text{Unique Links}}{\text{Total Links}}
    $$
   - Return the result as a float rounded to three decimal places.

## 🌐 Link Diversity Metric

The Link Diversity metric is used to measure the variety of sources within a collection of links. It is calculated as the ratio of the number of unique links to the total number of links observed.


### Interpretation

* **High Link Diversity (closer to 1):** Indicates a wide variety of sources, where most of the links point to different destinations.
* **Low Link Diversity (closer to 0):** Suggests that a few specific links are being repeated often, indicating a high concentration or reliance on a small number of sources.
**Example Usage:**

```python
html_content = '''
<html><body>
<a href="http://siteA.com">Link A</a>
<a href="http://siteA.com">Link A again</a>
<a href="http://siteB.com">Link B</a>
</body></html>
'''
diversity = calculate_link_diversity(html_content)
print("Link Diversity:", diversity)
```
---

In [26]:
def calculate_link_diversity(html):
    bs = BeautifulSoup(html, 'html.parser')
    urls = bs.find_all('a', href=True)

    # filter out inbound urls
    non_inB_urls = []
    for url in urls:
        href = url['href'].strip()
        
        if not href:
            continue
        if href.startswith('#') or href.startswith('/'):
            continue
        non_inB_urls.append(href.lower())

    if len(non_inB_urls)==0:
        return 0.0

    # diversity
    unique_links = len(set(non_inB_urls))
    total_links = len(non_inB_urls)

    diversity = unique_links/total_links
    return round(diversity,3)

In [27]:
html = '''
<html><body>
<a href="http://siteA.com">Link A</a>
<a href="http://siteA.com">Link A again</a>
<a href="http://siteB.com">Link B</a>
</body></html>
'''
diversity = calculate_link_diversity(html)
print("Link Diversity:", diversity)

Link Diversity: 0.667


### Q14. Reflection and Extension

Write a structured summary of your findings for each of the questions from **Q6 to Q13**. For each technique or function you implemented, address the following aspects:

- **Weaknesses of the technique**  
  Identify limitations, edge cases, or scenarios where the technique may fail or produce unreliable results.

- **Strengths of the technique**  
  Highlight what the technique does well, including its efficiency, simplicity, or effectiveness in typical use cases.

- **Improvement opportunities**  
  Explain how addressing the identified weaknesses could enhance the technique’s accuracy, scalability, or robustness.

**Instructions:**

- Format your answers as separate paragraphs for each question (Q6 to Q13).
- Use headings or bold labels to clearly identify each question.
- Ensure your report includes meaningful insights and examples from your implementation.
- Use markdown formatting to keep your notebook organized and readable.

---

#### Q6
Keyword stuffing is a good technique to find inflated term frequencies given a keyword. This approach is very strong in considering which words are inflated given a threshold. There are some limitations to this however, such as long documents may be penalized due to their natural occurence of words that may appear a lot due to the fact that the document itself is large. An approach top fixing this weakness may be to implement a ratio of the term frequncy by the overall length of the document, and if this ratio exceeds a certain threshold, we can say that the keyword appears unnaturally in the document.

#### Q7
Finding out concealed keywords in css is useful due to the fact that we can find attributes that are invisible to users by parsed by crawlers. The algorithm is fast to execute and is good at catching common concelement techniques such as "display:none" and "opacity:0". A weakness this approach might have is that it can probably be evaded by creating more complex css strings, which would require more complex rules and domain knowledge by the developers trying to catch these techniques. One improvement may be to use regex patterns to account for similar hidden texts but with slight variations.


#### Q8
Cloaking detection is good for catching text which is different for the user and for the crawler, and using a similarity score to find how much they differ by. A weakness of this approach would be to access the crawler view which does not sound feasable in every scenario. Location based sites which change per region may be affected by the crawler view since it might not account for the different regions. One way to improve this algorithm might be to check html strings for more details on the document.

#### Q9
Doorway detection does a good job of finding common redirection patterns in html strings, it is fast in retrieval and efficient. There may be false positives in this scenario due to the algorithm catching very basic patterns, and a lot of legitimate sites may use redirects (such as mobile apps or portal logins). One improvement may be to only flag redirects which take 0 seconds, as that may be suspicious due to most notable websites giving a notice of redirect with a 5-second rate of redirection.

#### Q10
Link spam detection is good at identifying artifical link patterns where the same URL is repeadetly used. It can use a threshold parameter which is adjustable by the user as well. The drawback to link spam detection is that it cannot properly catch link spam accross many coordinated sites, link quality is also not taken into consideration, since a high quality webpage may be used multiple times in the current webpage. This algorithm can be heavily improved by combining it with the content quality algorithm of links that are in the html tag.

#### Q11
Content quality is good in terms that it uses ratios such as the average sentence length, stopwords, and unique vocabulary richness to estimate the quality of a page. The weakness this algorithm has is that it may be biased towards a certain type of document. Since the rules are very hard-coded, the content quality estimation may for example rank healthcare documents higher than sports documents due to the fact that it has more unique words, and less stopwords. One improvement for this will be to have an algorithm to estimate thresholds (such as average sentence length), for different document topics such as sports, healthcare, film, etc. 

This algotihm cannot also detect factual accuracy of a document which may be an issue due to the fact that innacurate documents should automatically be lower in quality. One fix could be to add source credibility to have a better estimate.

#### Q12
Keyword density is good in terms of that it uses a ratio of term frequency by the length of the document to find inflated terms, rather than just the number of occurences by itself. A weakness is that it does not understand the semantic context of a document, and also it does not catch synonyms, or variations of a word (such a running, sprint, sprinting, and run are all similar). It can be improved using a thesauri and synonym expansion to find a more accurate score for the keyword density.

#### Q13
Link diversity of a webpage score is a good way of finding the link diversity of a webpage, by parsing the html tag and finding inflated links. It makes it easy to find pages that are repeats and spam urls. it cannot find the quality of the link however, and it may create false positives from verified links that have been cited multiple times in the webpage. We can improve this with our content estimation algorithm, by estimating the cited url's quality, and then flagging any occuring urls that have a low quality score.